Instalar MySQL, descargar driver y cargar datos de BBDD de empleados
https://dev.mysql.com/doc/employee/en/

i. Cargar con spark datos de empleados y departamentos

ii. Mediante Joins mostrar toda la información de los empleados además de su
título y salario

In [4]:
from pyspark.sql import SparkSession
#create a SparkSession\n"
spark = (SparkSession
        .builder
        .config("spark.sql.catalogImplementation","hive")
        .config("spark.jars", "/media/jose/Repositorio/spark3/jars/mysql-connector-java-8.0.26.jar")
        .master("local")
        .appName("Example-3_6")
        .getOrCreate())

### Abrir consola y ejecutar
   sudo mysql -u root -p
   
   pasword:$Iruelas2010
   
   escribir SET GLOBAL time_zone = '-3:00'; en mysql

In [69]:
current_dept_emp = (spark
                    .read
                    .format("jdbc")
                    .option("url", "jdbc:mysql://localhost:3306/employees")
                    .option("driver", "com.mysql.jdbc.Driver")
                    .option("dbtable", "current_dept_emp")                    
                    .option("user", "root")
                    .option("password", "$Iruelas2010")
                    .load()
                   )

current_dept_emp.show(5)

+------+-------+----------+----------+
|emp_no|dept_no| from_date|   to_date|
+------+-------+----------+----------+
| 10001|   d005|1986-06-26|9999-01-01|
| 10002|   d007|1996-08-03|9999-01-01|
| 10003|   d004|1995-12-03|9999-01-01|
| 10004|   d004|1986-12-01|9999-01-01|
| 10005|   d003|1989-09-12|9999-01-01|
+------+-------+----------+----------+
only showing top 5 rows



In [70]:
departments = (spark
               .read
               .format("jdbc")
               .option("url", "jdbc:mysql://localhost:3306/employees") 
               .option("driver", "com.mysql.jdbc.Driver")
               .option("dbtable", "departments")
               .option("user", "root")
               .option("password", "$Iruelas2010")
               .load()
              )

departments.show(5)

+-------+----------------+
|dept_no|       dept_name|
+-------+----------------+
|   d009|Customer Service|
|   d005|     Development|
|   d002|         Finance|
|   d003| Human Resources|
|   d001|       Marketing|
+-------+----------------+
only showing top 5 rows



In [71]:
dept_emp = (spark
            .read
            .format("jdbc")
            .option("url", "jdbc:mysql://localhost:3306/employees") 
            .option("driver", "com.mysql.jdbc.Driver")
            .option("dbtable", "dept_emp") 
            .option("user", "root")
            .option("password", "$Iruelas2010")
            .load()
           )

dept_emp.show(5)

+------+-------+----------+----------+
|emp_no|dept_no| from_date|   to_date|
+------+-------+----------+----------+
| 10001|   d005|1986-06-26|9999-01-01|
| 10002|   d007|1996-08-03|9999-01-01|
| 10003|   d004|1995-12-03|9999-01-01|
| 10004|   d004|1986-12-01|9999-01-01|
| 10005|   d003|1989-09-12|9999-01-01|
+------+-------+----------+----------+
only showing top 5 rows



In [72]:
dept_emp_latest_date = (spark
                        .read
                        .format("jdbc")
                        .option("url", "jdbc:mysql://localhost:3306/employees") 
                        .option("driver", "com.mysql.jdbc.Driver")
                        .option("dbtable", "dept_emp_latest_date")  
                        .option("user", "root")
                        .option("password", "$Iruelas2010")
                        .load()
                       )

dept_emp_latest_date.show(5)

+------+----------+----------+
|emp_no| from_date|   to_date|
+------+----------+----------+
| 10001|1986-06-26|9999-01-01|
| 10002|1996-08-03|9999-01-01|
| 10003|1995-12-03|9999-01-01|
| 10004|1986-12-01|9999-01-01|
| 10005|1989-09-12|9999-01-01|
+------+----------+----------+
only showing top 5 rows



In [73]:
dept_manager = (spark
                .read
                .format("jdbc")
                .option("url", "jdbc:mysql://localhost:3306/employees") 
                .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "dept_manager") 
                .option("user", "root")
                .option("password", "$Iruelas2010")
                .load()
               )

dept_manager.show(5)

+------+-------+----------+----------+
|emp_no|dept_no| from_date|   to_date|
+------+-------+----------+----------+
|110022|   d001|1985-01-01|1991-10-01|
|110039|   d001|1991-10-01|9999-01-01|
|110085|   d002|1985-01-01|1989-12-17|
|110114|   d002|1989-12-17|9999-01-01|
|110183|   d003|1985-01-01|1992-03-21|
+------+-------+----------+----------+
only showing top 5 rows



In [74]:
employees = (spark
             .read
             .format("jdbc")
             .option("url", "jdbc:mysql://localhost:3306/employees") 
             .option("driver", "com.mysql.jdbc.Driver")
             .option("dbtable", "employees") 
             .option("user", "root")
             .option("password", "$Iruelas2010")
             .load()
            )

employees.cache()
employees.show(5)

+------+----------+----------+---------+------+----------+
|emp_no|birth_date|first_name|last_name|gender| hire_date|
+------+----------+----------+---------+------+----------+
| 10001|1953-09-02|    Georgi|  Facello|     M|1986-06-26|
| 10002|1964-06-02|   Bezalel|   Simmel|     F|1985-11-21|
| 10003|1959-12-03|     Parto|  Bamford|     M|1986-08-28|
| 10004|1954-05-01| Chirstian|  Koblick|     M|1986-12-01|
| 10005|1955-01-21|   Kyoichi| Maliniak|     M|1989-09-12|
+------+----------+----------+---------+------+----------+
only showing top 5 rows



In [75]:
salaries = (spark.read.format("jdbc")
            .option("url", "jdbc:mysql://localhost:3306/employees") 
            .option("driver", "com.mysql.jdbc.Driver")
            .option("dbtable", "salaries") 
            .option("user", "root")
            .option("password", "$Iruelas2010")
            .load()
           )

salaries.cache()
salaries.show(5)

+------+------+----------+----------+
|emp_no|salary| from_date|   to_date|
+------+------+----------+----------+
| 10001| 60117|1986-06-26|1987-06-26|
| 10001| 62102|1987-06-26|1988-06-25|
| 10001| 66074|1988-06-25|1989-06-25|
| 10001| 66596|1989-06-25|1990-06-25|
| 10001| 66961|1990-06-25|1991-06-25|
+------+------+----------+----------+
only showing top 5 rows



In [76]:
titles = (spark
          .read
          .format("jdbc")
          .option("url", "jdbc:mysql://localhost:3306/employees") 
          .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "titles") 
          .option("user", "root")
          .option("password", "$Iruelas2010")
          .load()
         )

titles.cache()
titles.show(5)

+------+---------------+----------+----------+
|emp_no|          title| from_date|   to_date|
+------+---------------+----------+----------+
| 10001|Senior Engineer|1986-06-26|9999-01-01|
| 10002|          Staff|1996-08-03|9999-01-01|
| 10003|Senior Engineer|1995-12-03|9999-01-01|
| 10004|       Engineer|1986-12-01|1995-12-01|
| 10004|Senior Engineer|1995-12-01|9999-01-01|
+------+---------------+----------+----------+
only showing top 5 rows



### Salario actual

In [79]:
import pyspark.sql.functions as F

salario = (salaries
           .select("emp_no", "salary","from_date")
           .groupby("emp_no")
           .max("salary")
           .alias("Salario")  
          )

salario = (salario
           .select("emp_no",
                   F.col("max(salary)").alias("Salario")
                  )
          ) 

salaries.unpersist()
salario.cache()
salario.show(5)
salario.count()

+------+-------+
|emp_no|Salario|
+------+-------+
| 10206|  71052|
| 10362|  54987|
| 10623|  86399|
| 10817|  78202|
| 11033|  75271|
+------+-------+
only showing top 5 rows



300024

### Categoria actual

In [80]:
cat = (titles
       .select("emp_no",
               "title",
               "from_date",
              )
       .groupBy("emp_no")
       .agg(
           F.max(F.to_timestamp("from_date").cast("long")).alias("date_long")
       )
      )

cat.cache()
cat.show(5) 
cat.count()

+------+---------+
|emp_no|date_long|
+------+---------+
| 10206|577404000|
| 10362|657500400|
| 10623|916354800|
| 10817|914626800|
| 11033|889830000|
+------+---------+
only showing top 5 rows



300024

In [81]:
categoria = (cat
             .join(titles, cat.emp_no == titles.emp_no)
             .select(cat.emp_no,"title")
             .where(
                 F.col("date_long") == F.to_timestamp("from_date").cast("long")
             )
            )

titles.unpersist()
cat.unpersist()
categoria.cache()
categoria.show(5)
categoria.count()

+------+----------------+
|emp_no|           title|
+------+----------------+
| 68924|           Staff|
|202465|           Staff|
| 37629|Technique Leader|
| 75332|    Senior Staff|
| 79479| Senior Engineer|
+------+----------------+
only showing top 5 rows



300024

### Join employes-salario

In [82]:
employees_salaries = (salario
                      .join(employees,employees.emp_no == salario.emp_no)
                      .select(
                          salario.emp_no,
                          employees.first_name,
                          employees.last_name, 
                          employees.birth_date,
                          employees.gender,
                          salario.Salario)
                     )

employees.unpersist()
salario.unpersist()
employees_salaries.cache().show(5)
employees_salaries.count()

+------+----------+---------+----------+------+-------+
|emp_no|first_name|last_name|birth_date|gender|Salario|
+------+----------+---------+----------+------+-------+
| 10206|  Alassane|  Iwayama|1960-09-19|     F|  71052|
| 10362|   Shalesh|  dAstous|1963-09-16|     M|  54987|
| 10623|Aleksander|   Danlos|1953-07-11|     F|  86399|
| 10817|       Uri|  Rullman|1958-10-02|     F|  78202|
| 11033|   Shushma|     Bahk|1957-03-01|     F|  75271|
+------+----------+---------+----------+------+-------+
only showing top 5 rows



300024

In [83]:
employees_salaries_tittle = (employees_salaries
                             .join(categoria, employees_salaries.emp_no == categoria.emp_no)
                             .select(employees_salaries.emp_no.alias("ID"),
                                     F.concat_ws(",",employees_salaries.last_name,
                                                 employees_salaries.first_name
                                                ).alias("Apellido,Nombre"), 
                                     employees.birth_date,
                                     employees.gender,                                                
                                     categoria.title.alias("Categoría"),
                                     employees_salaries.Salario
                                    )
                            )

employees_salaries.unpersist()
employees_salaries_tittle.cache()  
employees_salaries.show(5)
employees_salaries_tittle.count()

+------+----------+---------+----------+------+-------+
|emp_no|first_name|last_name|birth_date|gender|Salario|
+------+----------+---------+----------+------+-------+
| 10206|  Alassane|  Iwayama|1960-09-19|     F|  71052|
| 10362|   Shalesh|  dAstous|1963-09-16|     M|  54987|
| 10623|Aleksander|   Danlos|1953-07-11|     F|  86399|
| 10817|       Uri|  Rullman|1958-10-02|     F|  78202|
| 11033|   Shushma|     Bahk|1957-03-01|     F|  75271|
+------+----------+---------+----------+------+-------+
only showing top 5 rows



300024

In [84]:
employees_salaries_tittle.orderBy("ID").show(5)

+-----+-----------------+----------+------+---------------+-------+
|   ID|  Apellido,Nombre|birth_date|gender|      Categoría|Salario|
+-----+-----------------+----------+------+---------------+-------+
|10001|   Facello,Georgi|1953-09-02|     M|Senior Engineer|  88958|
|10002|   Simmel,Bezalel|1964-06-02|     F|          Staff|  72527|
|10003|    Bamford,Parto|1959-12-03|     M|Senior Engineer|  43699|
|10004|Koblick,Chirstian|1954-05-01|     M|Senior Engineer|  74057|
|10005| Maliniak,Kyoichi|1955-01-21|     M|   Senior Staff|  94692|
+-----+-----------------+----------+------+---------------+-------+
only showing top 5 rows



In [85]:
employees_salaries_tittle.orderBy(F.col("Salario").asc()).show(5)

+-----+-----------------+----------+------+----------------+-------+
|   ID|  Apellido,Nombre|birth_date|gender|       Categoría|Salario|
+-----+-----------------+----------+------+----------------+-------+
|69048|  Matheson,Masoud|1952-05-26|     F|        Engineer|  40000|
|39285|   Pauthner,Marlo|1958-06-15|     M|Technique Leader|  40000|
|77422|Schiper,Mitsuyuki|1956-09-17|     F|        Engineer|  40000|
|63964| Gips,Przemyslawa|1963-12-31|     M|Technique Leader|  40000|
|94851|    Kusalik,Tonny|1961-07-11|     M|Technique Leader|  40000|
+-----+-----------------+----------+------+----------------+-------+
only showing top 5 rows



In [86]:
employees_salaries_tittle.orderBy(F.col("Salario").desc()).show(5)

+------+-----------------+----------+------+------------+-------+
|    ID|  Apellido,Nombre|birth_date|gender|   Categoría|Salario|
+------+-----------------+----------+------+------------+-------+
| 43624|   Pesch,Tokuyasu|1953-11-14|     M|Senior Staff| 158220|
|254466|Mukaidono,Honesty|1963-05-27|     M|Senior Staff| 156286|
| 47978|  Whitcomb,Xiahua|1956-03-24|     M|Senior Staff| 155709|
|253939|    Luders,Sanjai|1957-12-03|     M|Senior Staff| 155513|
|109334|Alameldin,Tsutomu|1955-08-02|     M|Senior Staff| 155377|
+------+-----------------+----------+------+------------+-------+
only showing top 5 rows



In [87]:
employees_salaries_tittle.orderBy("Apellido,Nombre").show(5)

+------+-----------------+----------+------+---------------+-------+
|    ID|  Apellido,Nombre|birth_date|gender|      Categoría|Salario|
+------+-----------------+----------+------+---------------+-------+
|258641|Aamodt,Abdelkader|1961-05-23|     M|Senior Engineer|  49813|
|258005|   Aamodt,Adhemar|1953-02-17|     F|       Engineer|  53566|
|455773|  Aamodt,Aemilian|1960-05-04|     M|   Senior Staff| 112566|
|436560|     Aamodt,Alagu|1959-03-16|     F|       Engineer|  62759|
|266651|Aamodt,Aleksander|1959-05-28|     F|Senior Engineer|  80318|
+------+-----------------+----------+------+---------------+-------+
only showing top 5 rows



In [88]:
employees_salaries_tittle.orderBy(F.col("Apellido,Nombre").desc()).show(5)

+------+----------------+----------+------+----------------+-------+
|    ID| Apellido,Nombre|birth_date|gender|       Categoría|Salario|
+------+----------------+----------+------+----------------+-------+
| 50521|    dAstous,Zito|1962-12-05|     F|    Senior Staff|  88807|
| 99107|dAstous,Zhonghui|1957-10-13|     M|    Senior Staff|  68777|
| 30234|   dAstous,Yurii|1960-02-07|     F| Senior Engineer|  68818|
| 94784|   dAstous,Yucai|1952-04-13|     F|Technique Leader|  40000|
|273557| dAstous,Younwoo|1962-05-19|     F|    Senior Staff|  73492|
+------+----------------+----------+------+----------------+-------+
only showing top 5 rows



In [89]:
employees_salaries_tittle.orderBy("birth_date").show(5)

+------+-----------------+----------+------+---------------+-------+
|    ID|  Apellido,Nombre|birth_date|gender|      Categoría|Salario|
+------+-----------------+----------+------+---------------+-------+
|207658|Whitcomb,Kiyokazu|1952-02-01|     M|Senior Engineer|  89770|
|237571|   Schaad,Ronghao|1952-02-01|     M|   Senior Staff|  59355|
| 87461|Decaestecker,Moni|1952-02-01|     M|Senior Engineer|  67914|
|406121|    Remmele,Supot|1952-02-01|     M|   Senior Staff|  76470|
| 65308|  Pocchiola,Jouni|1952-02-01|     M|   Senior Staff|  65562|
+------+-----------------+----------+------+---------------+-------+
only showing top 5 rows



In [90]:
employees_salaries_tittle.orderBy(F.col("birth_date").desc()).show(5)

+------+------------------+----------+------+---------------+-------+
|    ID|   Apellido,Nombre|birth_date|gender|      Categoría|Salario|
+------+------------------+----------+------+---------------+-------+
|470472|Staudhammer,Zhiguo|1965-02-01|     F|Senior Engineer|  62226|
|270792|  Wendorf,Domenico|1965-02-01|     M|       Engineer|  44388|
|457832| Zielinski,Jackson|1965-02-01|     F|Senior Engineer|  44526|
|449984|     Perin,Hinrich|1965-02-01|     F|Senior Engineer|  69425|
|287323|       Dengi,Steve|1965-02-01|     M|          Staff|  78448|
+------+------------------+----------+------+---------------+-------+
only showing top 5 rows



### Media de sueldos por genero

In [91]:
(employees_salaries_tittle
 .groupBy("gender")
 .agg(F.avg("Salario"))
 .show()
)

+------+-----------------+
|gender|     avg(Salario)|
+------+-----------------+
|     F|69940.39245820526|
|     M|69967.90245759086|
+------+-----------------+



### numero de trabajadores por genero

In [92]:
(employees_salaries_tittle
 .groupBy("gender")
 .count()
 .show()
)

+------+------+
|gender| count|
+------+------+
|     F|120051|
|     M|179973|
+------+------+



### numero de trabajadores por categoria 

In [93]:
(employees_salaries_tittle
 .groupBy("Categoría")
 .count()
 .show()
)

+------------------+-----+
|         Categoría|count|
+------------------+-----+
|             Staff|41129|
|Assistant Engineer| 5835|
|      Senior Staff|92846|
|   Senior Engineer|97747|
|          Engineer|47303|
|           Manager|    9|
|  Technique Leader|15155|
+------------------+-----+



### numero de trabajadores por categoria y genero

In [94]:
(employees_salaries_tittle
 .groupBy("Categoría").pivot("gender")
 .count()
 .withColumn("Total", F.col("F") + F.col("M"))
 .show()
)

+------------------+-----+-----+-----+
|         Categoría|    F|    M|Total|
+------------------+-----+-----+-----+
|Assistant Engineer| 2333| 3502| 5835|
|             Staff|16418|24711|41129|
|      Senior Staff|37082|55764|92846|
|   Senior Engineer|39139|58608|97747|
|          Engineer|18963|28340|47303|
|           Manager|    4|    5|    9|
|  Technique Leader| 6112| 9043|15155|
+------------------+-----+-----+-----+



### numero de empleados con sueldo superior o igual a 100000

In [95]:
(employees_salaries_tittle
 .select("*")
 .where(F.col("Salario") >= 100000)
 .count()
)

19021

### numero de empleados con sueldo superior o igual a 500000 e inferior a 100000

In [96]:
(employees_salaries_tittle
 .select("*")
 .where((F.col("Salario") < 100000) & (F.col("Salario") >= 50000))
 .count()
)

243359

### por debajo de 50000

In [97]:
(employees_salaries_tittle
 .select("*")
 .where(F.col("Salario") < 50000)
 .count()
 )

37644

### Mayores de 65

In [98]:
(employees_salaries_tittle
 .select("*")
 .where(F.col("birth_date") <= "1956-10-08")
 .count()
)

108170

### Entre 60 y 65

In [99]:
(employees_salaries_tittle
 .select("*")
 .where((F.col("birth_date") > "1956-10-08") & (F.col("birth_date") <= "1961-10-08"))
 .count()
)

115594

### Entre 56 y 60

In [100]:
(employees_salaries_tittle
 .select("*")
 .where((F.col("birth_date") > "1961-10-08") & (F.col("birth_date") <= "1965-02-01"))
 .count()
)

76260

### Menos de 55

In [101]:
(employees_salaries_tittle
 .select("*")
 .where((F.col("birth_date") > "1966-10-08"))
 .count()
)

0

In [102]:
employees_salaries_tittle.unpersist()

DataFrame[ID: int, Apellido,Nombre: string, birth_date: date, gender: string, Categoría: string, Salario: int]